In [2]:
import torch
import numpy as np
import pandas as pd
import json, nltk


In [4]:
device=torch.device("cuda")
device

device(type='cuda')

In [5]:
data = pd.read_csv('dataset.csv', encoding='cp1252')
data

,question_text
0,How are mountains formed?
1,What causes the formation of mountains?
2,Explain the geological processes behind mounta...
3,Can you describe the process by which mountain...
4,I'm unclear about the origin of mountains – ca...
...,...
280,What is the role of the United Nations in addr...
281,How do ocean trenches and seafloor spreading r...
282,Describe the impact of ocean pollution on mari...
283,What is the importance of the Danube River in ...


In [6]:
with open('contractions.json', 'r') as f:
    contractions_dict = json.load(f)
contractions = contractions_dict['contractions']
contractions

{"ain't": 'am not / are not',
 "aren't": 'are not / am not',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "'cause": 'because',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he had / he would',
 "he'd've": 'he would have',
 "he'll": 'he shall / he will',
 "he'll've": 'he shall have / he will have',
 "he's": 'he has / he is',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how has / how is',
 "i'd": 'I had / I would',
 "i'd've": 'I would have',
 "i'll": 'I shall / I will',
 "i'll've": 'I shall have / I will have',
 "i'm": 'I am',
 "i've": 'I have',
 "isn't": 'is not',
 "it'd": 'it had / it would',
 "it'd've": 'it would have',
 "it'll": 'it shall / it will',
 "it'll've": 'it shall have / it will have',
 "it's": 'it has / it is',
 "let's": 'l

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (stopwords and WordNet corpus) if you haven't already
nltk.download('stopwords')
nltk.download('wordnet')

def process_tweet(text):
    text = text.lower()  # Lowercases the string
    text = re.sub(r"\d+", " ", str(text))  # Removes all digits
    text = re.sub(r"\b[a-zA-Z]\b", "", str(text))  # Removes all single characters
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])  # Replaces contractions
    text = re.sub(r"[^\w\s]", " ", str(text))  # Removes all punctuations
    text = re.sub(r'(.)\1+', r'\1\1', text)  # Convert more than 2 letter repetitions to 2 letter
    text = re.sub(r"\s+", " ", str(text))  # Replaces double spaces with single space
    
    # Tokenize the text
    words = text.split()
    
    # Remove stopwords and perform lemmatization
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    # Join the words back into a sentence
    text = " ".join(words)
    
    return text

# Example usage:
# tweet = "Your input tweet goes here."
# processed_tweet = process_tweet(tweet)
# print(processed_tweet)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
data['preprocessed_ques'] = data['question_text'].apply(process_tweet)

In [13]:
data

,question_text,preprocessed_ques
0,How are mountains formed?,mountain formed
1,What causes the formation of mountains?,cause formation mountain
2,Explain the geological processes behind mounta...,explain geological process behind mountain for...
3,Can you describe the process by which mountain...,describe process mountain created
4,I'm unclear about the origin of mountains – ca...,unclear origin mountain explain
...,...,...
280,What is the role of the United Nations in addr...,role united nation addressing climate change
281,How do ocean trenches and seafloor spreading r...,ocean trench seafloor spreading relate plate t...
282,Describe the impact of ocean pollution on mari...,describe impact ocean pollution marine life
283,What is the importance of the Danube River in ...,importance danube river europe


In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:
data

,question_text,preprocessed_ques
0,How are mountains formed?,mountain formed
1,What causes the formation of mountains?,cause formation mountain
2,Explain the geological processes behind mountain formation.,explain geological process behind mountain formation
3,Can you describe the process by which mountains are created?,describe process mountain created
4,I'm unclear about the origin of mountains – can you explain?,unclear origin mountain explain
...,...,...
280,What is the role of the United Nations in addressing climate change?,role united nation addressing climate change
281,How do ocean trenches and seafloor spreading relate to plate tectonics?,ocean trench seafloor spreading relate plate tectonics
282,Describe the impact of ocean pollution on marine life.,describe impact ocean pollution marine life
283,What is the importance of the Danube River in Europe?,importance danube river europe


In [19]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (stopwords and WordNet corpus) if you haven't already
nltk.download('stopwords')
nltk.download('wordnet')

def process_ques_without_stopwords(text):
    text = text.lower()  # Lowercases the string
    text = re.sub(r"\d+", " ", str(text))  # Removes all digits
    text = re.sub(r"\b[a-zA-Z]\b", "", str(text))  # Removes all single characters
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])  # Replaces contractions
    text = re.sub(r"[^\w\s]", " ", str(text))  # Removes all punctuations
    text = re.sub(r'(.)\1+', r'\1\1', text)  # Convert more than 2 letter repetitions to 2 letter
    text = re.sub(r"\s+", " ", str(text))  # Replaces double spaces with single space
    
    # Tokenize the text
    words = text.split()
    
    # Remove stopwords and perform lemmatization
    # stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the words back into a sentence
    text = " ".join(words)
    
    return text

# Example usage:
# tweet = "Your input tweet goes here."
# processed_tweet = process_tweet(tweet)
# print(processed_tweet)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
data['preprocessed_ques_without_stopwords'] = data['question_text'].apply(process_ques_without_stopwords)

In [21]:
data

,question_text,preprocessed_ques,preprocessed_ques_without_stopwords
0,How are mountains formed?,mountain formed,how are mountain formed
1,What causes the formation of mountains?,cause formation mountain,what cause the formation of mountain
2,Explain the geological processes behind mountain formation.,explain geological process behind mountain formation,explain the geological process behind mountain formation
3,Can you describe the process by which mountains are created?,describe process mountain created,can you describe the process by which mountain are created
4,I'm unclear about the origin of mountains – can you explain?,unclear origin mountain explain,unclear about the origin of mountain can you explain
...,...,...,...
280,What is the role of the United Nations in addressing climate change?,role united nation addressing climate change,what is the role of the united nation in addressing climate change
281,How do ocean trenches and seafloor spreading relate to plate tectonics?,ocean trench seafloor spreading relate plate tectonics,how do ocean trench and seafloor spreading relate to plate tectonics
282,Describe the impact of ocean pollution on marine life.,describe impact ocean pollution marine life,describe the impact of ocean pollution on marine life
283,What is the importance of the Danube River in Europe?,importance danube river europe,what is the importance of the danube river in europe


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000)

In [27]:
tfidf_preprocessed_ques = tfidf.fit_transform(data['preprocessed_ques']).toarray()
tfidf_preprocessed_ques_without_stopwords = tfidf.fit_transform(data['preprocessed_ques_without_stopwords']).toarray()


In [31]:
tfidf_preprocessed_ques

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
tfidf_preprocessed_ques_without_stopwords


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
## DBSCAN on preprocessed_ques
from sklearn.cluster import DBSCAN

# Apply DBSCAN clustering to the TF-IDF vectors
# You might need to tune the epsilon (eps) and min_samples parameters based on your data
dbscan = DBSCAN(eps=0.5, min_samples=5)
cluster_labels = dbscan.fit_predict(tfidf_preprocessed_ques)

# Add the cluster labels to the DataFrame
data['cluster'] = cluster_labels

# List down questions in different clusters
unique_clusters = set(cluster_labels)
for cluster in unique_clusters:
    if cluster == -1:
        # -1 represents noise (points that do not belong to any cluster)
        print("Noise:")
    else:
        print("Cluster {}:".format(cluster))
    questions_in_cluster = data[data['cluster'] == cluster]['question_text']
    print(questions_in_cluster)
    print("\n---\n")


Cluster 0:
45     Can you describe the impact of urban sprawl on land use?
114            Describe the impact of urban sprawl on land use.
140            Describe the impact of urban sprawl on land use.
158            Describe the impact of urban sprawl on land use.
184            Describe the impact of urban sprawl on land use.
202            Describe the impact of urban sprawl on land use.
228            Describe the impact of urban sprawl on land use.
246            Describe the impact of urban sprawl on land use.
272            Describe the impact of urban sprawl on land use.
Name: question_text, dtype: object

---

Cluster 1:
49     Explain the importance of the Mississippi River in the United States.
111    What is the importance of the Mississippi River in the United States?
155    What is the importance of the Mississippi River in the United States?
199    What is the importance of the Mississippi River in the United States?
243    What is the importance of the Mississippi Rive

In [40]:
count = 0
noise = 0
for cluster in unique_clusters:
    if cluster == -1:
        # -1 represents noise (points that do not belong to any cluster)
        noise_questions = data[data['cluster'] == -1]['question_text']
        noise= noise+1
        print("noise_questions")
    else:
        # print("Cluster {}:".format(cluster))
        # Get the first question in the cluster
        question_in_cluster = data[data['cluster'] == cluster]['question_text'].iloc[0]
        print(question_in_cluster)
        count = count+1
        # print("\n---\n")

# Print the count and format it in the desired output form
print("Number of cluster = {}".format(count))
print("Number of noise = {}".format(noise))


Can you describe the impact of urban sprawl on land use?
What is the significance of the Amazon River in South America?
Explain the importance of the Mississippi River in the United States.
How do fjords develop in coastal regions?
Explain the concept of a continental divide.
How do geological fault lines lead to earthquakes?
Describe the impact of ocean pollution on marine life.
How do ocean trenches and seafloor spreading relate to plate tectonics?
What is the role of the United Nations in addressing climate change?
Explain the significance of the Great Wall of China.
How do atolls form in oceanic environments?
Explain the concept of the Dust Bowl in the United States.
What is the impact of the greenhouse effect on sea levels?
What is the importance of the Danube River in Europe?
noise_questions
Number of cluster = 14
Number of noise = 1


In [35]:
## DBSCAN on preprocessed_ques_without_stopwords
from sklearn.cluster import DBSCAN

# Apply DBSCAN clustering to the TF-IDF vectors
# You might need to tune the epsilon (eps) and min_samples parameters based on your data
dbscan = DBSCAN(eps=0.5, min_samples=5)
cluster_labels = dbscan.fit_predict(tfidf_preprocessed_ques_without_stopwords)

# Add the cluster labels to the DataFrame
data['cluster'] = cluster_labels

# List down questions in different clusters
unique_clusters = set(cluster_labels)
for cluster in unique_clusters:
    if cluster == -1:
        # -1 represents noise (points that do not belong to any cluster)
        print("Noise:")
    else:
        print("Cluster {}:".format(cluster))
    questions_in_cluster = data[data['cluster'] == cluster]['question_text']
    print(questions_in_cluster)
    print("\n---\n")


Cluster 0:
45     Can you describe the impact of urban sprawl on land use?
114            Describe the impact of urban sprawl on land use.
140            Describe the impact of urban sprawl on land use.
158            Describe the impact of urban sprawl on land use.
184            Describe the impact of urban sprawl on land use.
202            Describe the impact of urban sprawl on land use.
228            Describe the impact of urban sprawl on land use.
246            Describe the impact of urban sprawl on land use.
272            Describe the impact of urban sprawl on land use.
Name: question_text, dtype: object

---

Cluster 1:
47     What is the significance of the Amazon River in South America?
123      What is the importance of the Amazon River in South America?
144      What is the importance of the Amazon River in South America?
167      What is the importance of the Amazon River in South America?
188      What is the importance of the Amazon River in South America?
211      Wh

In [37]:
count =0
for cluster in unique_clusters:
    if cluster == -1:
        # -1 represents noise (points that do not belong to any cluster)
        print("Noise:")
    else:
        print("Cluster {}:".format(cluster))
        # Get the first question in the cluster
        question_in_cluster = data[data['cluster'] == cluster]['question_text'].iloc[0]
        print(question_in_cluster)
        count = count+1
        print("\n---\n")

# Print the count and format it in the desired output form
print("Number of unique questions = {}".format(count))


Cluster 0:
Can you describe the impact of urban sprawl on land use?

---

Cluster 1:
What is the significance of the Amazon River in South America?

---

Cluster 2:
Explain the importance of the Mississippi River in the United States.

---

Cluster 3:
How do fjords develop in coastal regions?

---

Cluster 4:
Explain the concept of a continental divide.

---

Cluster 5:
How do geological fault lines lead to earthquakes?

---

Cluster 6:
Describe the impact of ocean pollution on marine life.

---

Cluster 7:
How do ocean trenches and seafloor spreading relate to plate tectonics?

---

Cluster 8:
What is the role of the United Nations in addressing climate change?

---

Cluster 9:
Explain the significance of the Great Wall of China.

---

Cluster 10:
How do atolls form in oceanic environments?

---

Cluster 11:
Explain the concept of the Dust Bowl in the United States.

---

Cluster 12:
What is the impact of the greenhouse effect on sea levels?

---

Cluster 13:
What is the importance of

In [ ]:
# Print the count and format it in the desired output form
print("Number of unique questions = {}".format(count))
